In [1]:
pip install yfinance mplfinance pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import random
import numpy as np
import pandas as pd
import yfinance as yf
import mplfinance as mpf
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

# Parameters
IMAGES_DIR = "balanced_stock_patterns"
os.makedirs(IMAGES_DIR, exist_ok=True)
STOCK_SYMBOLS = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "META", "NVDA", "TSLA", "NFLX",
    "JPM", "BAC", "V", "MA", "SOFI", "COIN",
    "XOM", "CVX", "BA", "CAT", "DE",
    "PFE", "JNJ", "MRK", "LLY",
    "PLTR", "SNOW", "SHOP", "RIVN", "UBER",
    "SPY", "QQQ"
]

START_DATE = "2019-01-01"
END_DATE = "2024-04-01"


# Initialize pattern counters
pattern_counts = {}

def detect_head_and_shoulders(data):
    closing_prices = data["Close"].values
    indices = np.arange(len(closing_prices))

    peaks = argrelextrema(closing_prices, np.greater, order=6)[0]
    troughs = argrelextrema(closing_prices, np.less, order=6)[0]

    if len(peaks) >= 3 and len(troughs) >= 1:
        for i in range(len(peaks) - 2):
            left_shoulder, head, right_shoulder = peaks[i], peaks[i + 1], peaks[i + 2]

            # Allow some variation in height
            shoulder_height_tolerance = 0.1 * closing_prices[head]

            if (
                closing_prices[left_shoulder] < closing_prices[head] and
                closing_prices[right_shoulder] < closing_prices[head] and
                abs(closing_prices[left_shoulder] - closing_prices[right_shoulder]) < shoulder_height_tolerance
            ):
                # Ensure the neckline exists
                relevant_troughs = [closing_prices[t] for t in troughs if left_shoulder < t < right_shoulder]
                if relevant_troughs and closing_prices[right_shoulder] < np.mean(relevant_troughs):
                    return "Head and Shoulders"

    return "None"


# Detect Double Top
def detect_double_top(data):
    closing_prices = data["Close"].values
    peaks = argrelextrema(closing_prices, np.greater, order=5)[0]

    if len(peaks) >= 2:
        for i in range(len(peaks) - 1):
            peak1, peak2 = peaks[i], peaks[i + 1]
            price_diff = abs(closing_prices[peak1] - closing_prices[peak2])
            avg_peak = (closing_prices[peak1] + closing_prices[peak2]) / 2

            if price_diff < 0.03 * avg_peak and (peak2 - peak1) >= 5:
                return "Double Top"
    return "None"

# Detect Double Bottom
def detect_double_bottom(data):
    closing_prices = data["Close"].values
    troughs = argrelextrema(closing_prices, np.less, order=5)[0]

    if len(troughs) >= 2:
        for i in range(len(troughs) - 1):
            t1, t2 = troughs[i], troughs[i + 1]
            diff = abs(closing_prices[t1] - closing_prices[t2])
            avg = (closing_prices[t1] + closing_prices[t2]) / 2

            if diff < 0.03 * avg and (t2 - t1) >= 5:
                return "Double Bottom"
    return "None"


def detect_cup_and_handle(data):
    closing_prices = data["Close"].values
    peaks = argrelextrema(closing_prices, np.greater, order=8)[0]
    troughs = argrelextrema(closing_prices, np.less, order=8)[0]

    if len(peaks) < 2 or len(troughs) < 1:
        return "None"

    for i in range(len(peaks) - 1):
        left_peak, right_peak = peaks[i], peaks[i + 1]
        relevant_troughs = [t for t in troughs if left_peak < t < right_peak]

        if relevant_troughs:
            min_trough = min(relevant_troughs, key=lambda x: closing_prices[x])
            drop_pct = (closing_prices[left_peak] - closing_prices[min_trough]) / closing_prices[left_peak]

            if 0.08 < drop_pct < 0.5 and (right_peak - left_peak) > 8:  # More flexible drop %
                return "Cup and Handle"

    return "None"

def detect_ascending_triangle(data):
    closing_prices = data["Close"].values
    peaks = argrelextrema(closing_prices, np.greater, order=6)[0]
    troughs = argrelextrema(closing_prices, np.less, order=6)[0]

    if len(peaks) < 3 or len(troughs) < 3:
        return "None"

    resistance_level = np.mean(closing_prices[peaks[-3:]])
    last_troughs = closing_prices[troughs[-3:]]

    if all(last_troughs[i] > last_troughs[i - 1] for i in range(1, len(last_troughs))):
        return "Ascending Triangle"

    return "None"

def detect_descending_triangle(data):
    closing_prices = data["Close"].values
    peaks = argrelextrema(closing_prices, np.greater, order=6)[0]
    troughs = argrelextrema(closing_prices, np.less, order=6)[0]

    if len(peaks) < 3 or len(troughs) < 3:
        return "None"

    support_level = np.mean(closing_prices[troughs[-3:]])
    last_peaks = closing_prices[peaks[-3:]]

    if all(last_peaks[i] < last_peaks[i - 1] for i in range(1, len(last_peaks))):
        return "Descending Triangle"

    return "None"

def detect_falling_wedge(data):
    closing_prices = data["Close"].values
    peaks = argrelextrema(closing_prices, np.greater, order=6)[0]
    troughs = argrelextrema(closing_prices, np.less, order=6)[0]

    if len(peaks) < 3 or len(troughs) < 3:
        return "None"

    last_peaks = closing_prices[peaks[-3:]]
    last_troughs = closing_prices[troughs[-3:]]

    if all(last_peaks[i] < last_peaks[i - 1] for i in range(1, len(last_peaks))) and \
       all(last_troughs[i] < last_troughs[i - 1] for i in range(1, len(last_troughs))):
        return "Falling Wedge"

    return "None"

def detect_rising_wedge(data):
    closing_prices = data["Close"].values
    peaks = argrelextrema(closing_prices, np.greater, order=6)[0]
    troughs = argrelextrema(closing_prices, np.less, order=6)[0]

    if len(peaks) < 3 or len(troughs) < 3:
        return "None"

    last_peaks = closing_prices[peaks[-3:]]
    last_troughs = closing_prices[troughs[-3:]]

    if all(last_peaks[i] > last_peaks[i - 1] for i in range(1, len(last_peaks))) and \
       all(last_troughs[i] > last_troughs[i - 1] for i in range(1, len(last_troughs))):
        return "Rising Wedge"

    return "None"

def detect_bullish_engulfing(data):
    for i in range(1, len(data) - 1):
        prev_candle = data.iloc[i - 1]
        curr_candle = data.iloc[i]

        # Ensure first candle is red (bearish) and second is green (bullish)
        if prev_candle["Close"] < prev_candle["Open"] and curr_candle["Close"] > curr_candle["Open"]:
            # Ensure the bullish candle completely engulfs the previous bearish candle
            if curr_candle["Close"] > prev_candle["Open"] and curr_candle["Open"] < prev_candle["Close"]:
                # Extra condition: Higher volume on engulfing candle increases reliability
                if curr_candle["Volume"] > prev_candle["Volume"]:
                    return "Bullish Engulfing"
    return "None"

def detect_bearish_engulfing(data):
    for i in range(1, len(data) - 1):
        prev_candle = data.iloc[i - 1]
        curr_candle = data.iloc[i]

        # Ensure first candle is green (bullish) and second is red (bearish)
        if prev_candle["Close"] > prev_candle["Open"] and curr_candle["Close"] < curr_candle["Open"]:
            # Ensure the bearish candle completely engulfs the previous bullish candle
            if curr_candle["Close"] < prev_candle["Open"] and curr_candle["Open"] > prev_candle["Close"]:
                # Extra condition: Higher volume on engulfing candle increases reliability
                if curr_candle["Volume"] > prev_candle["Volume"]:
                    return "Bearish Engulfing"
    return "None"

def detect_hammer(data, min_lower_shadow_ratio=2.0):
    for i in range(2, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        open_price = candle["Open"]
        close_price = candle["Close"]
        high = candle["High"]
        low = candle["Low"]

        body = abs(close_price - open_price)
        lower_shadow = min(open_price, close_price) - low
        upper_shadow = high - max(open_price, close_price)

        # Basic hammer shape
        if lower_shadow > min_lower_shadow_ratio * body and upper_shadow < body * 0.3:
            # Context check: downtrend before this
            if prev_candle["Close"] < data.iloc[i - 2]["Close"] < candle["Close"]:
                return "Hammer"

    return "None"

def detect_inverted_hammer(data, min_upper_shadow_ratio=2.0):
    for i in range(2, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        open_price = candle["Open"]
        close_price = candle["Close"]
        high = candle["High"]
        low = candle["Low"]

        body = abs(close_price - open_price)
        upper_shadow = high - max(open_price, close_price)
        lower_shadow = min(open_price, close_price) - low

        # Inverted hammer shape
        if upper_shadow > min_upper_shadow_ratio * body and lower_shadow < body * 0.3:
            # Context check: previous candles should be in downtrend
            if prev_candle["Close"] < data.iloc[i - 2]["Close"] < candle["Close"]:
                return "Inverted Hammer"

    return "None"

def detect_hanging_man(data, min_lower_shadow_ratio=2.0):
    for i in range(2, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        open_price = candle["Open"]
        close_price = candle["Close"]
        high = candle["High"]
        low = candle["Low"]

        body = abs(close_price - open_price)
        lower_shadow = min(open_price, close_price) - low
        upper_shadow = high - max(open_price, close_price)

        # Hanging Man shape
        if lower_shadow > min_lower_shadow_ratio * body and upper_shadow < body * 0.3:
            # Context: Uptrend before this
            if prev_candle["Close"] > data.iloc[i - 2]["Close"] < candle["Close"]:
                return "Hanging Man"

    return "None"

def detect_shooting_star(data, min_upper_shadow_ratio=2.0):
    for i in range(2, len(data)):
        candle = data.iloc[i]
        prev_candle = data.iloc[i - 1]

        open_price = candle["Open"]
        close_price = candle["Close"]
        high = candle["High"]
        low = candle["Low"]

        body = abs(close_price - open_price)
        upper_shadow = high - max(open_price, close_price)
        lower_shadow = min(open_price, close_price) - low

        # Shooting Star shape
        if upper_shadow > min_upper_shadow_ratio * body and lower_shadow < body * 0.3:
            # Context: Uptrend before this
            if prev_candle["Close"] > data.iloc[i - 2]["Close"] < candle["Close"]:
                return "Shooting Star"

    return "None"

def detect_morning_star(data):
    for i in range(2, len(data)):
        c1 = data.iloc[i - 2]  # First (bearish)
        c2 = data.iloc[i - 1]  # Doji or small
        c3 = data.iloc[i]      # Bullish

        # Candle 1: Strong bearish
        if c1["Close"] < c1["Open"] and abs(c1["Open"] - c1["Close"]) > 0.5 * (c1["High"] - c1["Low"]):
            # Candle 2: Small real body
            if abs(c2["Open"] - c2["Close"]) <= 0.3 * abs(c1["Open"] - c1["Close"]):
                # Candle 3: Strong bullish and closes > midpoint of candle 1
                if c3["Close"] > c3["Open"] and c3["Close"] > (c1["Open"] + c1["Close"]) / 2:
                    return "Morning Star"

    return "None"

def detect_evening_star(data):
    for i in range(2, len(data)):
        c1 = data.iloc[i - 2]  # First (bullish)
        c2 = data.iloc[i - 1]  # Doji or small
        c3 = data.iloc[i]      # Bearish

        # Candle 1: Strong bullish
        if c1["Close"] > c1["Open"] and abs(c1["Close"] - c1["Open"]) > 0.5 * (c1["High"] - c1["Low"]):
            # Candle 2: Small real body
            if abs(c2["Open"] - c2["Close"]) <= 0.3 * abs(c1["Close"] - c1["Open"]):
                # Candle 3: Strong bearish and closes below midpoint of candle 1
                if c3["Close"] < c3["Open"] and c3["Close"] < (c1["Open"] + c1["Close"]) / 2:
                    return "Evening Star"

    return "None"


def detect_patterns(data):
    for detector in [
        detect_head_and_shoulders,
        detect_double_top,
        detect_double_bottom,
        detect_cup_and_handle,
        detect_ascending_triangle,
        detect_descending_triangle,
        detect_falling_wedge,
        detect_rising_wedge,
        detect_bullish_engulfing,
        detect_bearish_engulfing,
        detect_hammer,
        detect_inverted_hammer,
        detect_hanging_man,
        detect_shooting_star,
        detect_morning_star,
        detect_evening_star,
    ]:
        pattern = detector(data)
        if pattern != "None":
            return pattern
    return "None"



# Fetch stock data
def fetch_stock_data(symbol, start, end):
    """Fetch stock data and ensure necessary columns exist."""
    stock = yf.download(symbol, start=start, end=end, auto_adjust=False, actions=False, repair=True)

    # Convert multi-index columns to single index
    if isinstance(stock.columns, pd.MultiIndex):
        stock.columns = stock.columns.get_level_values(0)

    # Drop NaNs to avoid mplfinance errors
    stock.dropna(subset=["Open", "High", "Low", "Close"], inplace=True)

    return stock


def create_pattern_folder(pattern_name):
    folder = os.path.join(IMAGES_DIR, pattern_name.replace(" ", "_"))
    os.makedirs(folder, exist_ok=True)
    return folder

def save_chart(data, folder, filename):
    fig, ax = plt.subplots(figsize=(8, 6))
    mpf.plot(data, type="candle", style="charles", ax=ax, volume=False)
    path = os.path.join(folder, filename)
    plt.savefig(path)
    plt.close()

print("🚀 Starting balanced dataset generation...")

annotations = []
TARGET_COUNT = 500 # For debugging
WINDOW_SIZE = 50
STEP_SIZE = 5

all_patterns = [
    "Head and Shoulders", "Double Top", "Double Bottom", "Cup and Handle", "Ascending Triangle",
    "Descending Triangle", "Falling Wedge", "Rising Wedge", "Bullish Engulfing", "Bearish Engulfing",
    "Hammer", "Inverted Hammer", "Hanging Man", "Shooting Star", "Morning Star", "Evening Star"
]
pattern_counts = {pattern: 0 for pattern in all_patterns}

print("🚀 Starting balanced dataset generation...")

MAX_ATTEMPTS =35  # Fail-safe: how many passes over all symbols before we stop even if not 100% filled
attempts = 0

while any(count < TARGET_COUNT for count in pattern_counts.values()) and attempts < MAX_ATTEMPTS:
    attempts += 1
    print(f"🔁 Pass {attempts} over symbols...")

    for symbol in STOCK_SYMBOLS:
        print(f"📈 Fetching data for {symbol}")
        stock_data = fetch_stock_data(symbol, START_DATE, END_DATE)

        for start_idx in range(0, len(stock_data) - WINDOW_SIZE, STEP_SIZE):
            subset = stock_data.iloc[start_idx:start_idx + WINDOW_SIZE]
            pattern = detect_patterns(subset)

            if pattern != "None" and pattern_counts.get(pattern, 0) < TARGET_COUNT:
                folder = create_pattern_folder(pattern)
                filename = f"{symbol}_{pattern_counts[pattern]}.png"
                save_chart(subset, folder, filename)

                annotations.append([filename, symbol, pattern])
                pattern_counts[pattern] += 1
                print(f"✅ {pattern}: {pattern_counts[pattern]}/{TARGET_COUNT}")

            # Early stop inside the window loop
            if all(c >= TARGET_COUNT for c in pattern_counts.values()):
                break

        if all(c >= TARGET_COUNT for c in pattern_counts.values()):
            break

print("🎯 Balanced dataset complete!")
df = pd.DataFrame(annotations, columns=["Filename", "Stock", "Pattern"])
df.to_csv("balanced_annotations.csv", index=False)



[*********************100%***********************]  1 of 1 completed

🚀 Starting balanced dataset generation...
🚀 Starting balanced dataset generation...
🔁 Pass 1 over symbols...
📈 Fetching data for AAPL


✅ Double Bottom: 1/500
✅ Double Bottom: 2/500
✅ Double Bottom: 3/500
✅ Double Bottom: 4/500
✅ Double Top: 1/500
✅ Double Bottom: 5/500
✅ Double Bottom: 6/500
✅ Hammer: 1/500
✅ Double Top: 2/500
✅ Double Top: 3/500
✅ Double Top: 4/500
✅ Double Top: 5/500
✅ Double Top: 6/500
✅ Double Top: 7/500
✅ Double Top: 8/500
✅ Double Top: 9/500
✅ Cup and Handle: 1/500
✅ Double Top: 10/500
✅ Bearish Engulfing: 1/500
✅ Double Top: 11/500
✅ Bearish Engulfing: 2/500
✅ Bearish Engulfing: 3/500
✅ Double Bottom: 7/500
✅ Double Top: 12/500
✅ Double Top: 13/500
✅ Double Top: 14/500
✅ Double Top: 15/500
✅ Double Top: 16/500
✅ Double Top: 17/500
✅ Double Top: 18/500
✅ Bullish Engulfing: 1/500
✅ Double Bottom: 8/500
✅ Bullish Engulfing: 2/500
✅ Bullish Engulfing: 3/500
✅ Double Bottom: 9/500
✅ Bullish Engulfing: 4/500
✅ Bullish Engulfing: 5/500
✅ Double Top: 19/500
✅ Double Top: 20/500
✅ Double Top: 21/500
✅ Double Top: 22/500
✅ Double Top: 23/500
✅ Double Top: 24/500
✅ Double Top: 25/500
✅ Double Top: 26/500


[*********************100%***********************]  1 of 1 completed

✅ Double Top: 132/500
📈 Fetching data for MSFT


...

✅ Descending Triangle: 418/500
📈 Fetching data for UBER


✅ Descending Triangle: 419/500
📈 Fetching data for SPY


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

📈 Fetching data for QQQ


✅ Hammer: 385/500
✅ Descending Triangle: 420/500
🎯 Balanced dataset complete!


In [1]:
import os

pattern_dirs = os.listdir("balanced_stock_patterns")  # or your actual IMAGES_DIR path
for pattern_dir in pattern_dirs:
    count = len(os.listdir(os.path.join("balanced_stock_patterns", pattern_dir)))
    print(f"{pattern_dir}: {count} images")

Ascending_Triangle: 1202 images
Bearish_Engulfing: 1907 images
Bullish_Engulfing: 1968 images
Cup_and_Handle: 1976 images
Descending_Triangle: 1136 images
Double_Bottom: 1896 images
Double_Top: 1949 images
Evening_Star: 710 images
Hammer: 885 images
Hanging_Man: 1113 images
Head_and_Shoulders: 1030 images
Inverted_Hammer: 535 images
Morning_Star: 1032 images
Shooting_Star: 953 images


In [12]:
import os
import pandas as pd

# Path to the root folder containing pattern subfolders
IMAGE_DIR = "balanced_stock_patterns"

# List to hold the annotation rows
annotations = []

# Traverse each pattern folder
for pattern_folder in os.listdir(IMAGE_DIR):
    pattern_path = os.path.join(IMAGE_DIR, pattern_folder)
    
    if os.path.isdir(pattern_path):
        # Convert folder name back to readable pattern name
        pattern_name = pattern_folder.replace("_", " ")
        
        for filename in os.listdir(pattern_path):
            if filename.endswith(".png"):
                stock_symbol = filename.split("_")[0]
                annotations.append([filename, stock_symbol, pattern_name])

# Create and save the annotations CSV
df = pd.DataFrame(annotations, columns=["Filename", "Stock", "Pattern"])
df.to_csv("annotations.csv", index=False)

print("annotations.csv file created with", len(df),"entries.")

annotations.csv file created with 18292 entries.
